In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from efficientnet_pytorch import EfficientNet

In [4]:
torch.cuda.is_available()

True

In [6]:
class DocumentClassifier(nn.Module):
    def __init__(self,num_classes):
        super(DocumentClassifier).__init__()
        
        self.backbone = EfficientNet.from_pretrained('efficientnet-b0')
        
        self.classificationhead=nn.Sequential(
            nn.Linear(30,10),
            nn.Linear(10,num_classes),
            self.Softmax(num_classes)
        )
    